<b>タチコマ tutorial
- https://zanote.net/ai/chatbot1/#%E3%82%B3%E3%83%BC%E3%83%89

kargs, args
- https://www.freecodecamp.org/news/args-and-kwargs-in-python/

save data via pickle
- https://www.robotech-note.com/entry/2016/10/01/180840

In [30]:
import transformers
import numpy as np
import pandas as pd
import os
import random
import time 
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter
import copy
from torch.optim.lr_scheduler import LambdaLR
import pickle


from transformers import (
    T5Tokenizer,
    GPT2LMHeadModel,
    GPT2DoubleHeadsModel,
    GPT2Tokenizer,
    AutoModel,
    BertJapaneseTokenizer,
    BertModel,
    AutoModelForSeq2SeqLM,
    BertForSequenceClassification,
    AutoModelForQuestionAnswering,
    BertForQuestionAnswering,
    AutoModelForCausalLM,
    OpenAIGPTDoubleHeadsModel,
    GPT2DoubleHeadsModel,
    OpenAIGPTTokenizer
)

In [33]:
df = pd.read_excel("japanese_persona_chat.xlsx", index_col=0)
df2 = pd.read_excel("japanese_persona_chat.xlsx", index_col=0,sheet_name=['ペルソナリスト'])

In [135]:
df.loc[1]

,ペルソナID,話者,発話
No,,,
1,PP1,A,こんにちは。お元気ですか？
1,PP1,B,はい、元気です。広告代理店での仕事が忙しいですが。
1,PP1,A,お疲れさまです。私は介護福祉士をしています。
1,PP1,B,介護福祉士として働いていらっしゃるんですね。大変なお仕事ですよね。
1,PP1,A,いえ。広告代理店も大変ですよね。ちなみに私は北海道の一軒家に住んでいます。
1,PP1,B,北海道ですか。ご飯の美味しいところで羨ましいです。私は青森県出身で山の多いところが好きなので...
1,PP1,A,北海道は、美味しいもの多いですよ。青森なら、お近くでしたね。でも、私はいつか東京に住んでみた...
1,PP1,B,東京ですか。東京には華やかな場所が多いですよね
1,PP1,A,田舎生まれの私にとって、とても魅力的な場所です。


In [34]:
def make_dataset(df):
    df = df
    def make_candidates(df):
        to_return = {}
        for i in range(len(set(df.index))):
            to_return[str(i+1)]=[]
            for j in range(len(df.loc[i+1])):
                if j%2!=0:
                    to_return[str(i+1)].append([df.loc[i+1].発話.values[j]])
        return to_return #type: dict {'1': ['はい、元気です。広告代理店での仕事が忙しいですが。', ...
    candidates = make_candidates(df)
    def make_history(df):
        to_return = {}
        for i in range(len(set(df.index))):
            to_return[str(i+1)]=[]
            for j in range(len(df.loc[i+1])):
                if j%2==0:
                    xx = []
                    for k in range(j+1):
                        xx.append(df.loc[i+1].発話.values[k])
                    to_return[str(i+1)].append(xx)
        return to_return
    history = make_history(df)
    return candidates,history



In [35]:
candidates, history= make_dataset(df=df)

In [39]:
candidates['1']

[['はい、元気です。広告代理店での仕事が忙しいですが。'],
 ['介護福祉士として働いていらっしゃるんですね。大変なお仕事ですよね。'],
 ['北海道ですか。ご飯の美味しいところで羨ましいです。私は青森県出身で山の多いところが好きなので、山の近くに住んでいます'],
 ['東京ですか。東京には華やかな場所が多いですよね'],
 ['私にとっても東京は魅力的な場所ですね。お休みの日は何をされていますか？私は写真が趣味です。'],
 ['1人でのんびりと過ごす時間も大切で、贅沢なものですよ。私もよく家でのんびり過ごしています。'],
 ['そうですね。仕事の息抜きに1人の時間も大切です。']]

<b>output: utterance,history:
- conversation id: 1~5000
  - conversation_length: 0~.

In [40]:
def make_utterances(candidates,history):
    utterances = []
    for i in range(1,len(candidates)+1):
        utterances.append([])
        for j in range(len(candidates[str(i)])):
            utterances[i-1].append(
            {
                "candidates": candidates[str(i)][j], "history":history[str(i)][j]
            }
            )
    return utterances

In [41]:
utterances = make_utterances(candidates,history)

In [51]:
utterances[:2]

[[{'candidates': ['はい、元気です。広告代理店での仕事が忙しいですが。'], 'history': ['こんにちは。お元気ですか？']},
  {'candidates': ['介護福祉士として働いていらっしゃるんですね。大変なお仕事ですよね。'],
   'history': ['こんにちは。お元気ですか？',
    'はい、元気です。広告代理店での仕事が忙しいですが。',
    'お疲れさまです。私は介護福祉士をしています。']},
  {'candidates': ['北海道ですか。ご飯の美味しいところで羨ましいです。私は青森県出身で山の多いところが好きなので、山の近くに住んでいます'],
   'history': ['こんにちは。お元気ですか？',
    'はい、元気です。広告代理店での仕事が忙しいですが。',
    'お疲れさまです。私は介護福祉士をしています。',
    '介護福祉士として働いていらっしゃるんですね。大変なお仕事ですよね。',
    'いえ。広告代理店も大変ですよね。ちなみに私は北海道の一軒家に住んでいます。']},
  {'candidates': ['東京ですか。東京には華やかな場所が多いですよね'],
   'history': ['こんにちは。お元気ですか？',
    'はい、元気です。広告代理店での仕事が忙しいですが。',
    'お疲れさまです。私は介護福祉士をしています。',
    '介護福祉士として働いていらっしゃるんですね。大変なお仕事ですよね。',
    'いえ。広告代理店も大変ですよね。ちなみに私は北海道の一軒家に住んでいます。',
    '北海道ですか。ご飯の美味しいところで羨ましいです。私は青森県出身で山の多いところが好きなので、山の近くに住んでいます',
    '北海道は、美味しいもの多いですよ。青森なら、お近くでしたね。でも、私はいつか東京に住んでみたいんです。']},
  {'candidates': ['私にとっても東京は魅力的な場所ですね。お休みの日は何をされていますか？私は写真が趣味です。'],
   'history': ['こんにちは。お元気ですか？',
    'はい、元気です。広告代理店での仕事が忙しいですが。',
    'お疲れさまです。私は介護

In [42]:
def make_personality(df):
    df2 = df
    personality = []
    for i in range(1,len(df2['ペルソナリスト'].B)+1):
        personality.append(df2['ペルソナリスト'].B[i].split('\n'))
    return personality

In [43]:
personality = make_personality(df2)

In [226]:
personality[:10]

[['私は青森で生まれました。',
  '私は山のそばに住んでいます。',
  '私は昭和生まれです。',
  '私は広告代理店勤務です。',
  '私の趣味は写真撮影です。'],
 ['私は以前、沖縄に住んでいたことがあります。',
  '私は海のそばに住んでいます。',
  '私はダンスが得意です。',
  '私は怒りっぽいです。',
  '私が尊敬する人は、母親です。'],
 ['私は外国生まれです。', '私が生まれたのは冬です。', '私の誕生日は、正月です。', '私は彫りの深い顔立ちです。', '私には兄がいます。'],
 ['私は秋田で生まれました。',
  '私は学校のそばに住んでいます。',
  '私は70年代の生まれです。',
  '私は教師です。',
  '私はチアリーダーをやっていました。'],
 ['私は新潟から出たことがありません。', '私は実家暮らしです。', '私は整体師です。', '私は足が短いです。', '私には弟がいます。'],
 ['私は12月生まれです。',
  '私の誕生日は、大みそかです。',
  '私の血液型はA型です。',
  '私は猫を飼っています。',
  '私は占いに興味があります。'],
 ['私は乙女座です。', '私は小学生です。', '私は占い師になりたいです。', '私は身長が低いです。', '私はニンジンが嫌いです。'],
 ['私は産休中です。', '私はお尻が大きいです。', '私は純和風の顔立ちです。', '私は既婚です。', '私の趣味は外国語の勉強です。'],
 ['私は定年退職しました。',
  '私は髪の毛が薄いです。',
  '私は虫歯が1本もないです。',
  '私は妻と熟年離婚しました。',
  '私は釣りが趣味です。'],
 ['私は岩手県で生まれました。',
  '私が好きな作家は、宮沢賢治です。',
  '私はネイリストになりたいです。',
  '私はエステティシャンです。',
  '私は熱帯魚を飼っています。']]

In [44]:
def make_data(personlaity,utterances):
    data = []
    for i in range(len(personality)):
        data.append({
            'personality': personality[i],
            'utterances': utterances[i] 
        })
    
    return data

In [52]:
data = make_data(personality,utterances)

In [53]:
data[0]

{'personality': ['私は青森で生まれました。',
  '私は山のそばに住んでいます。',
  '私は昭和生まれです。',
  '私は広告代理店勤務です。',
  '私の趣味は写真撮影です。'],
 'utterances': [{'candidates': ['はい、元気です。広告代理店での仕事が忙しいですが。'],
   'history': ['こんにちは。お元気ですか？']},
  {'candidates': ['介護福祉士として働いていらっしゃるんですね。大変なお仕事ですよね。'],
   'history': ['こんにちは。お元気ですか？',
    'はい、元気です。広告代理店での仕事が忙しいですが。',
    'お疲れさまです。私は介護福祉士をしています。']},
  {'candidates': ['北海道ですか。ご飯の美味しいところで羨ましいです。私は青森県出身で山の多いところが好きなので、山の近くに住んでいます'],
   'history': ['こんにちは。お元気ですか？',
    'はい、元気です。広告代理店での仕事が忙しいですが。',
    'お疲れさまです。私は介護福祉士をしています。',
    '介護福祉士として働いていらっしゃるんですね。大変なお仕事ですよね。',
    'いえ。広告代理店も大変ですよね。ちなみに私は北海道の一軒家に住んでいます。']},
  {'candidates': ['東京ですか。東京には華やかな場所が多いですよね'],
   'history': ['こんにちは。お元気ですか？',
    'はい、元気です。広告代理店での仕事が忙しいですが。',
    'お疲れさまです。私は介護福祉士をしています。',
    '介護福祉士として働いていらっしゃるんですね。大変なお仕事ですよね。',
    'いえ。広告代理店も大変ですよね。ちなみに私は北海道の一軒家に住んでいます。',
    '北海道ですか。ご飯の美味しいところで羨ましいです。私は青森県出身で山の多いところが好きなので、山の近くに住んでいます',
    '北海道は、美味しいもの多いですよ。青森なら、お近くでしたね。でも、私はいつか東京に住んでみたいんです。']},
  {'candidates': ['私にとっても東京

In [72]:
import pickle
def fetch_data(Dir):
    f = open(Dir,'rb')
    list_row = pickle.load(f)
    return list_row
def save_data(Dir,data):
    f = open(Dir, 'wb')
    list_row = data
    pickle.dump(list_row, f)

In [73]:
Dir1 = 'japanese_persona_dataset_mod.txt'
Dir2 = 'can_his.txt'

In [74]:
save_data(Dir1,data)

In [75]:
dataset = fetch_data(Dir1)

In [76]:
dataset[:2]

[{'personality': ['私は青森で生まれました。',
   '私は山のそばに住んでいます。',
   '私は昭和生まれです。',
   '私は広告代理店勤務です。',
   '私の趣味は写真撮影です。'],
  'utterances': [{'candidates': ['はい、元気です。広告代理店での仕事が忙しいですが。'],
    'history': ['こんにちは。お元気ですか？']},
   {'candidates': ['介護福祉士として働いていらっしゃるんですね。大変なお仕事ですよね。'],
    'history': ['こんにちは。お元気ですか？',
     'はい、元気です。広告代理店での仕事が忙しいですが。',
     'お疲れさまです。私は介護福祉士をしています。']},
   {'candidates': ['北海道ですか。ご飯の美味しいところで羨ましいです。私は青森県出身で山の多いところが好きなので、山の近くに住んでいます'],
    'history': ['こんにちは。お元気ですか？',
     'はい、元気です。広告代理店での仕事が忙しいですが。',
     'お疲れさまです。私は介護福祉士をしています。',
     '介護福祉士として働いていらっしゃるんですね。大変なお仕事ですよね。',
     'いえ。広告代理店も大変ですよね。ちなみに私は北海道の一軒家に住んでいます。']},
   {'candidates': ['東京ですか。東京には華やかな場所が多いですよね'],
    'history': ['こんにちは。お元気ですか？',
     'はい、元気です。広告代理店での仕事が忙しいですが。',
     'お疲れさまです。私は介護福祉士をしています。',
     '介護福祉士として働いていらっしゃるんですね。大変なお仕事ですよね。',
     'いえ。広告代理店も大変ですよね。ちなみに私は北海道の一軒家に住んでいます。',
     '北海道ですか。ご飯の美味しいところで羨ましいです。私は青森県出身で山の多いところが好きなので、山の近くに住んでいます',
     '北海道は、美味しいもの多いですよ。青森なら、お近くでしたね。でも、私はいつか東京に住んでみたいんです。']},
  

<b>json dumps list of dic to json file
- https://www.geeksforgeeks.org/python-convert-list-of-dictionaries-to-json/

In [77]:
import json
with open("dataset.json", "w") as final:
    json.dump(dataset, final)

In [78]:
g = open("dataset.json")
data = json.load(g)

In [79]:
type(data)

list

In [80]:
len(data)

5000

In [ ]:
data[:2]